In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

In [4]:
data = pd.read_csv(
    "market_basket_dataset.csv",
    sep=";",
)

C:\Users\athar\AppData\Local\Temp\ipykernel_53064\3684778503.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(


In [5]:
data.sample(5)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
464018,577469,GLASS SONGBIRD STORAGE JAR,1,20.11.2011 11:25,"12,5",15009.0,United Kingdom
65249,541827,FOLDING UMBRELLA RED/WHITE POLKADOT,3,21.01.2011 17:05,"3,29",NaN,United Kingdom
247800,559507,HANGING HEART ZINC T-LIGHT HOLDER,1,08.07.2011 15:22,"0,85",15920.0,United Kingdom
145825,549546,VINTAGE SEASIDE JIGSAW PUZZLES,3,10.04.2011 11:24,"3,75",16889.0,United Kingdom
215978,556489,FRENCH LAVENDER SCENT HEART,3,12.06.2011 13:59,"0,75",14704.0,United Kingdom


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [7]:
data["Price"] = (
    data["Price"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)


In [8]:
data.sample(5)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
71112,542303,BREAD BIN DINER STYLE IVORY,1,27.01.2011 11:05,16.95,12539.0,Spain
492762,579508,FRENCH LAUNDRY SIGN BLUE METAL,1,29.11.2011 16:33,3.29,NaN,United Kingdom
223907,557299,LOVE BUILDING BLOCK WORD,3,19.06.2011 13:56,5.95,13047.0,United Kingdom
64931,541811,LARGE HEART MEASURING SPOONS,2,21.01.2011 15:01,3.29,NaN,United Kingdom
314291,565541,MAGIC DRAWING SLATE DOLLY GIRL,1,05.09.2011 12:00,0.42,14159.0,United Kingdom
